In [ ]:
import gzip
import json
import string
import collections
from gensim.models import Word2Vec
import torch
import torch.nn as nn
import random
import torch.optim as optim

import nltk
from nltk.corpus import stopwords

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
f = gzip.open('/content/drive/MyDrive/CSE256_summary/cs_dataset/renttherunway_final_data.json.gz')
dataset = []
cnt=0
for l in f:
  dataset.append(json.loads(l))

In [ ]:
dataset[0]

{'fit': 'fit',
 'user_id': '420272',
 'bust size': '34d',
 'item_id': '2260466',
 'weight': '137lbs',
 'rating': '10',
 'rented for': 'vacation',
 'review_text': "An adorable romper! Belt and zipper were a little hard to navigate in a full day of wear/bathroom use, but that's to be expected. Wish it had pockets, but other than that-- absolutely perfect! I got a million compliments.",
 'body type': 'hourglass',
 'review_summary': 'So many compliments!',
 'category': 'romper',
 'height': '5\' 8"',
 'size': 14,
 'age': '28',
 'review_date': 'April 20, 2016'}

In [ ]:
raw_reviews = []
raw_summaries = []
for data in dataset:
  raw_reviews.append(data['review_text'])
  raw_summaries.append(data['review_summary'])
print(raw_reviews[0])
print(raw_summaries[0])

An adorable romper! Belt and zipper were a little hard to navigate in a full day of wear/bathroom use, but that's to be expected. Wish it had pockets, but other than that-- absolutely perfect! I got a million compliments.
So many compliments!


In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
class wv:
    def __init__(self):
        self.word2index = {"<SOS>":0, "<EOS>":1, "<PAD>":2} # word -> index
        self.word2count = {} # word -> frequency. This is only used for statistical work.
        self.index2word = {0:"<SOS>", 1:"<EOS>", 2:"<PAD>"} # index -> word
        self.n_words = 3 # <sos>, <eos>, <pad>

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
alltokens = [['<start>'], ['<end>'], ['<pad>']]
reviews = []
summaries = []
punctuations = set(string.punctuation)
stop_words = set(stopwords.words('english'))

for i in range(len(raw_reviews)):
  r = raw_reviews[i]
  s = raw_summaries[i]

  r = ''.join([c for c in r.lower() if c not in punctuations])
  s = ''.join([c for c in s.lower() if c not in punctuations])
  #print(r)
  #print(s)

  tokens = []
  for word in r.split(' '):
    if (word not in punctuations) and (word != ''):
      #print(word)
      word = word.lower()
      tokens.append(word)
  alltokens.append(list(tokens))
  reviews.append(' '.join(tokens))

  tokens = []
  for word in s.split(' '):
    if (word not in punctuations) and (word != ''):
      #print(word)
      word = word.lower()
      tokens.append(word)
  alltokens.append(list(tokens))
  summaries.append(' '.join(tokens))

In [ ]:
reviews[3]

'i rented this for my companys black tie awards banquet i liked that this dress was short but was a little fancier with the sequins i generally dont care for long dresses i would describe the color as more rose gold than yellow gold i have blonde hair and fair skin and the color was very flattering this is a very forgiving dress its form fitting without making you feel like youre squeezed into it i would rent this one again highly recommend'

In [ ]:
summaries[3]

'dress arrived on time and in perfect condition'

In [ ]:
source = wv()
target = wv()
pairs = []
for s, t in zip(reviews, summaries):
  if len(s.split(' ')) <50: continue
  source.addSentence(s)
  target.addSentence(t)
  full=[s,t]
  pairs.append(full)
  # break

In [ ]:
pairs[100]

['i wore this dress to my school of pharmacy centennial gala it definitely stood out and i received so many compliments from colleagues and strangers many commented that it was their favorite dress of the evening the dress was in excellent condition and the color was gorgeous the back was my favorite part i usually wear a size 2 but the 0 fit like a glove plenty of room and hugs every curve i loved this dress and would recommend it to anyone',
 'absolutely gorgeous compliments all night long']

In [ ]:
len(pairs)

96126

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils import data
from torch.utils.data import DataLoader
import os
import gensim
import string
from torch.nn import Embedding

class Summary_Loader(data.Dataset):
    def __init__(self, pairs, source, target,
                 review_length = 100, summary_length = 20):
        super().__init__()

        # self.reviews = reviews
        # self.summaries = summaries
        self.pairs = pairs

        self.review_length = review_length
        self.summary_length = summary_length

        self.source = source
        self.target = target

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, index):

        review = self.pairs[index][0].split(' ')
        summary = self.pairs[index][1].split(' ')
        # print(summary)

        review_tensor = np.zeros(shape = (self.review_length))
        summary_tensor = np.zeros(shape = (self.summary_length + 1))

        for i in range(self.review_length):
          if i < len(review):
            review_tensor[i] = self.source.word2index[review[i]]
          else:
            review_tensor[i] = self.source.word2index["<PAD>"]

        summary_tensor[0] = self.target.word2index["<SOS>"]
        for i in range(1, self.summary_length + 1):
          if i - 1 < len(summary):
            summary_tensor[i] = self.target.word2index[summary[i - 1]]
          elif i - 1 == len(summary):
            summary_tensor[i] = self.target.word2index["<EOS>"]
          else:
            summary_tensor[i] = self.target.word2index["<PAD>"]

        return torch.tensor(review_tensor).long(), torch.tensor(summary_tensor).long()


In [ ]:
summary_loader = Summary_Loader(pairs, source, target)

In [ ]:
from tqdm import tqdm

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_dim, emb_dim)

        self.rnn = nn.RNN(emb_dim, hid_dim, n_layers, dropout = dropout)

        self.dropout = nn.Dropout(dropout)

    def forward(self, src):

        embedded = self.dropout(self.embedding(src))

        outputs, hidden = self.rnn(embedded)

        return hidden

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()

        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.RNN(emb_dim, hid_dim, n_layers, dropout = dropout)

        self.fc = nn.Linear(hid_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden):

        input = input.unsqueeze(0)

        embedded = self.dropout(self.embedding(input))

        output, hidden = self.rnn(embedded,hidden)

        prediction = self.fc(output.squeeze(0))

        return prediction, hidden

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio = 0.5):

        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        hidden = self.encoder(src)

        input = trg[0,:]

        for t in range(1, trg_len):

            output, hidden = self.decoder(input, hidden)

            outputs[t] = output

            teacher_force = random.random() < teacher_forcing_ratio

            top1 = output.argmax(1)

            input = trg[t] if teacher_force else top1

        return outputs

In [ ]:
!pip install einops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.1 MB/s eta 0:00:00


In [ ]:
from tqdm import tqdm
import torch.optim as optim
from einops import rearrange
PAD_ID = 2

In [ ]:
# INPUT_DIM = len(SRC.vocab)
# OUTPUT_DIM = len(TRG.vocab)
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
INPUT_DIM = len(source.word2index)
OUTPUT_DIM = len(target.word2index)
print(INPUT_DIM, OUTPUT_DIM)

ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 4
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

trainset, valset = torch.utils.data.random_split(summary_loader, [0.95, 0.05], torch.Generator().manual_seed(42))
print(len(trainset), len(valset))
trainloader = DataLoader(trainset, batch_size = 128, shuffle = True, num_workers = 0)
valloader = DataLoader(valset, batch_size = 1, shuffle = True, num_workers = 0)

model = Seq2Seq(enc, dec, device).to(device)
# model = torch.compile(model)
optimizer = optim.Adam(model.parameters())

criterion = nn.CrossEntropyLoss(ignore_index = PAD_ID)
trainloader = DataLoader(dataset= summary_loader, batch_size= 256, shuffle= True, num_workers = 0, drop_last=True)

50892 11474
91320 4806


In [ ]:
ckpt = torch.load("/content/drive/MyDrive/CSE256_summary/models/RNN_v1.pkl", map_location='cpu')
model.load_state_dict(ckpt['model'])

<All keys matched successfully>

In [ ]:
def online_eval(model, trainloader):
  model = model.eval()
  tmp=[]
  tmp1=[]
  tmp2 =[]
  with tqdm(total = len(trainloader)) as pbar:
      for i, batch in enumerate(trainloader):
        input, label = batch
        input, label = input.to(device), label.to(device)
        # assert input.shape == (32, 216, 100) and label.shape == (32, 30, 100)
        input, label = rearrange(input, 'b l -> l b'), rearrange(label, 'b l -> l b')
        pred = model(input, label)

        pred = pred.permute(1, 0, 2)

        # pred = pred.cpu().detach().numpy()

        one_sample = pred[0]

        sf = nn.Softmax(dim = -1)
        one_sample = sf(one_sample)

        #print(one_sample.shape)
        one_sample = torch.argmax(one_sample, dim = -1)

        #print(one_sample.shape)
        #print(one_sample)

        one_sample = one_sample.cpu().detach().numpy()
        word = []
        for i in range(20):
          w = target.index2word[one_sample[i]]
          word.append(w)

        print("translate", word)

        label = rearrange(label, 'l b -> b l')
        word_label = []
        one_label = label.cpu().detach().numpy()[0]
        #print(one_label.shape)
        for i in range(20):
          w = target.index2word[one_label[i]]
          word_label.append(w)
        print("ground truth", word_label)
        pbar.update(1)
        # print(input.shape, label.shape)
        break

In [ ]:
def save(path, model):
  ckpt = {
      'model' : model.state_dict(),
  }
  torch.save(ckpt, path)
  print("save {} successfully".format(path))

In [ ]:
def train(model, iterator, optimizer, criterion, clip = None):

    model.train()

    epoch_loss = 0

    for e in range(100):
      with tqdm(total = len(iterator), unit = 'batch') as pbar:
        for i, batch in enumerate(iterator):
            model.train()
            src, trg = batch
            src, trg = rearrange(src, 'b l -> l b'), rearrange(trg, 'b l -> l b')
            src, trg = src.to(device), trg.to(device)
            # print(src.shape, trg.shape)

            optimizer.zero_grad()

            output = model(src, trg)

            output_dim = output.shape[-1]

            output = output[1:].reshape(-1, output_dim)
            trg = trg[1:].reshape(-1)

            loss = criterion(output, trg)

            loss.backward()

            optimizer.step()

            epoch_loss += loss.item()

            pbar.set_postfix(loss=loss.item())
            pbar.update(1)

      save("/content/drive/MyDrive/CSE256_summary/models/RNN_v1.pkl", model)
      print('Finish epoch ', e)
      print("eval...training")
      online_eval(model = model, trainloader = iterator)
      print("eval...validation")
      online_eval(model = model, trainloader = valloader)

In [ ]:
train(model, trainloader, optimizer, criterion )

100%|██████████| 375/375 [02:15<00:00,  2.77batch/s, loss=5.14]


save /content/drive/MyDrive/CSE256_summary/models/RNN_v1.pkl successfully
Finish epoch  0
eval...training


  0%|          | 1/375 [00:00<01:04,  5.78it/s]


translate ['<SOS>', 'i', 'dress', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>']
ground truth ['<SOS>', 'did', 'not', 'end', 'up', 'wearing', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
eval...validation


  0%|          | 1/4806 [00:00<01:51, 43.07it/s]


...

save /content/drive/MyDrive/CSE256_summary/models/RNN_v1.pkl successfully
Finish epoch  99
eval...training


  0%|          | 1/375 [00:00<01:08,  5.49it/s]


translate ['<SOS>', 'beautiful', 'dress', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>']
ground truth ['<SOS>', 'i', 'got', 'a', 'ton', 'of', 'compliments', 'on', 'this', 'dress', 'i', 'wore', 'it', 'for', 'a', 'june', 'wedding', 'in', 'a', 'marina']
eval...validation


  0%|          | 1/4806 [00:00<01:29, 53.89it/s]

translate ['<SOS>', 'beautiful', 'dress', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>']
ground truth ['<SOS>', 'very', 'flattering', 'style', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']


In [ ]:
! pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
model = model.eval()
online_eval(model = model, trainloader= trainloader)


In [ ]:
from rouge import Rouge

hypothesis = "this wore this dress a formal wedding and it the night away"

reference = "i wore this to a beach wedding and danced all night"

rouger = Rouge()
scores = rouger.get_scores(hypothesis, reference)

In [ ]:
scores

[{'rouge-1': {'r': 0.5454545454545454,
   'p': 0.5454545454545454,
   'f': 0.5454545404545454},
  'rouge-2': {'r': 0.2, 'p': 0.18181818181818182, 'f': 0.1904761854875285},
  'rouge-l': {'r': 0.5454545454545454,
   'p': 0.5454545454545454,
   'f': 0.5454545404545454}}]

In [ ]:
def eval_all(model, trainloader):
  model = model.eval()
  tmp=[]
  tmp1=[]
  tmp2 =[]
  score_r, score_p, score_f = 0,0,0
  num = 0
  with tqdm(total = len(trainloader)) as pbar:
      for j, batch in enumerate(trainloader):
        input, label = batch
        input, label = input.to(device), label.to(device)
        # assert input.shape == (32, 216, 100) and label.shape == (32, 30, 100)
        input, label = rearrange(input, 'b l -> l b'), rearrange(label, 'b l -> l b')
        pred = model(input, label)

        pred = pred.permute(1, 0, 2)

        # pred = pred.cpu().detach().numpy()

        one_sample = pred[0]

        sf = nn.Softmax(dim = -1)
        one_sample = sf(one_sample)

        #print(one_sample.shape)
        one_sample = torch.argmax(one_sample, dim = -1)

        #print(one_sample.shape)
        #print(one_sample)

        one_sample = one_sample.cpu().detach().numpy()
        word = []
        for i in range(20):
          w = target.index2word[one_sample[i]]
          word.append(w)

        filtered_word = []
        for i in range(1, len(word)):
          # For LSTM model, the first word is "<SOS>"
          if word[i] != "<EOS>":
            filtered_word.append(word[i])
          else:
            break
        pred_sentence = ' '.join(filtered_word)


        label = rearrange(label, 'l b -> b l')
        word_label = []
        one_label = label.cpu().detach().numpy()[0]
        #print(one_label.shape)
        for i in range(20):
          w = target.index2word[one_label[i]]
          word_label.append(w)

        filtered_label = []
        for i in range(1, len(word_label)):
          # For LSTM model, the first word is "<SOS>"
          if word_label[i] != "<EOS>":
            filtered_label.append(word_label[i])
          else:
            break
        target_sentence = ' '.join(filtered_label)

        if pred_sentence == '' or target_sentence == '':
          continue

        rouger = Rouge()
        scores = rouger.get_scores(pred_sentence, target_sentence)

        score_r += scores[0]['rouge-1']['r']
        score_p += scores[0]['rouge-1']['p']
        score_f += scores[0]['rouge-1']['f']
        num += 1

        # print(word)
        # print(word_label)
        # print(pred_sentence)
        # print(target_sentence)

        # print(scores)
        # print(scores[0]['rouge-1']['r'])
        # print(scores[0]['rouge-1']['p'])
        # print(scores[0]['rouge-1']['f'])

        pbar.update(1)
        # print(i)
        # if j == 1000:
        #   break
        # print(input.shape, label.shape)
        # break
      print(score_r / num, score_p / num, score_f / num)
      #print(score_r / 1000, score_p / 1000, score_f / 1000)
      return score_r, score_p, score_f



In [ ]:
model = model.eval()
# trainloader = DataLoader(dataset= summary_loader, batch_size= 1, shuffle= True, num_workers = 0, drop_last=True)
r,p,f = eval_all(model, valloader)

100%|█████████▉| 4797/4806 [01:09<00:00, 68.83it/s]

0.0986476940606789 0.2599541380029185 0.13124085095519755
